In [ ]:
# %% [markdown]
# # 医疗文献相关性分析工具 (基于开源 Chinese-BERT-WWM)
#
# **目标:** 基于用户输入的疾病名称，利用本地加载的 Chinese-BERT WWM 模型对论文摘要和标题进行零样本分类，给出相关性评级（高、中、低）。
#
# **运行环境:** 建议在 Jupyter Notebook 或 VS Code Notebook 中运行。
#
# **❗重要提示:** # 1. 本方法使用 Hugging Face 的 Zero-Shot Classification Pipeline。
# 2. 首次运行需下载模型，需耐心等待。
# 3. 推荐使用 GPU 运行以获得可接受的速度。
#
# **依赖库安装:**
# 请在运行前确保安装了以下库：
# ```bash
# # 安装 PyTorch (根据您的系统可能不同，请参考 PyTorch 官网)
# # pip install torch
#
# pip install pandas tqdm transformers
# ```

# %%
import json
import pandas as pd
import time
from tqdm import tqdm
import os
import random
from typing import List, Dict, Any

# 导入 Hugging Face 库
from transformers import pipeline

# --- 模型配置 ---
# 推荐使用 Zero-Shot Classification 管道
CLASSIFIER = None
# 已修正模型名称。原 'uer/chinese-roberta-wwm-ext-large' 不存在，改为 HFL 维护的标准大规模中文 RoBERTa 模型
MODEL_NAME = "hfl/chinese-roberta-wwm-ext-large"
LABELS = ["高", "中", "低"] # 预设的分类标签
# 用于零样本分类的假设模板（可以根据疾病调整）
# 例如: "这篇文章是关于 [疾病名称] 的。"
HYPOTHESIS_TEMPLATE = "这篇文章与疾病 {disease} 的关联度是{}"

# --- 全局配置 ---
FILE_PATH = 'papers.json'
OUTPUT_FILE_NAME = 'analyzed_papers_bert.csv'

# %% [markdown]
# ## 1. 初始化模型
#
# 加载 Hugging Face 的 Zero-Shot Classification Pipeline。

# %%
def initialize_classifier():
    """初始化并加载本地模型和 Pipeline。"""
    global CLASSIFIER
    print(f"--- 1. 正在加载模型: {MODEL_NAME} ---")
    try:
        # 使用 zero-shot-classification 管道
        # 首次运行时会自动下载模型
        CLASSIFIER = pipeline(
            "zero-shot-classification",
            model=MODEL_NAME,
            device=-1 # -1 for CPU, 0 or greater for GPU index
        )
        print("模型加载成功，设备设置为 CPU/GPU。")
    except Exception as e:
        print(f"致命错误: 模型加载失败。请检查 'transformers' 和 'torch' 是否正确安装。错误: {e}")
        CLASSIFIER = None

# %% [markdown]
# ## 2. BERT 本地分类函数
#
# 定义函数，使用加载的模型进行相关性分类。

# %%
def classify_relevance_with_bert(paper_title: str, paper_abstract: str, disease_name: str) -> Dict[str, str]:
    """
    使用 Hugging Face 的零样本分类 Pipeline 评估论文与疾病的相关性。
    """
    if not CLASSIFIER:
        return {"relevance": "N/A", "reason": "模型未初始化"}

    # 将标题和摘要拼接起来作为输入文本
    text_to_classify = f"标题: {paper_title}。摘要: {paper_abstract}"

    # 零样本分类调用
    try:
        # 分类结果包含每个标签的得分
        result = CLASSIFIER(
            text_to_classify,
            LABELS,
            hypothesis_template=HYPOTHESIS_TEMPLATE.format(disease=disease_name),
            multi_label=False # 确保只选择一个最高标签
        )

        # 提取最高得分的标签作为相关性结果
        relevance = result['labels'][0]
        score = result['scores'][0]

        # 构建理由
        reason = f"基于零样本分类，模型认为该文本与 '{relevance}' 的相似度最高 (置信度: {score:.2f})。"

        return {"relevance": relevance, "reason": reason}

    except Exception as e:
        # 模型推理失败（可能是输入过长等原因）
        return {"relevance": "N/A", "reason": f"BERT 推理失败: {e}"}


# %% [markdown]
# ## 3. 数据加载与分析主流程
#
# 定义主函数来处理数据并调用本地分类模型。

# %%
def analyze_papers_bert(file_path: str):
    """
    主函数：加载数据、获取用户输入、调用BERT模型分析并保存结果。
    """

    # 初始化模型
    initialize_classifier()
    if not CLASSIFIER:
        return

    print(f"\n--- 1. 正在尝试加载文件: {file_path} ---")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        df = pd.DataFrame(data)
        print(f"成功加载 {len(df)} 篇文献数据。")

    except FileNotFoundError:
        print(f"错误: 找不到文件 '{file_path}'。请确保文件存在于当前目录下。")
        return
    except json.JSONDecodeError:
        print(f"错误: 文件 '{file_path}' 不是一个有效的 JSON 格式。")
        return

    # 获取用户输入
    disease_name = input("\n--- 2. 请输入您想查找的疾病名称 (例如: 原发性血小板减少症): ")
    if not disease_name:
        print("未输入疾病名称，分析终止。")
        return

    print(f"\n--- 3. 正在对 {len(df)} 篇文献进行 BERT 零样本分析... ---")

    relevance_results: List[Dict[str, Any]] = []

    # 使用 tqdm 显示进度条
    for index, row in tqdm(df.iterrows(), total=len(df), desc="分析进度"):

        paper_title = row['title']
        paper_abstract = row['abstract']

        # 调用 BERT 分类函数
        result = classify_relevance_with_bert(
            paper_title=paper_title,
            paper_abstract=paper_abstract,
            disease_name=disease_name
        )

        # 将分析结果添加到原始数据行中
        result_row = row.to_dict()
        result_row[f'{disease_name}_相关性'] = result['relevance']
        result_row[f'{disease_name}_评估理由'] = result['reason']
        relevance_results.append(result_row)

    # 结果整合
    df_results = pd.DataFrame(relevance_results)

    # 结果排序 (可选)
    relevance_order = {'高': 3, '中': 2, '低': 1, 'N/A': 0}
    sort_column = f'{disease_name}_相关性'
    if sort_column in df_results.columns:
        df_results['sort_key'] = df_results[sort_column].apply(lambda x: relevance_order.get(x, 0))
        df_results = df_results.sort_values(by='sort_key', ascending=False).drop(columns='sort_key')


    # --- 4. 结果展示与保存 ---

    print("\n--- 4. 分析结果展示 (前5条) ---")

    display_cols = ['id', 'title', 'abstract', f'{disease_name}_相关性', f'{disease_name}_评估理由']
    actual_display_cols = [col for col in display_cols if col in df_results.columns]

    print(df_results[actual_display_cols].head())

    # 保存为 CSV
    df_results.to_csv(OUTPUT_FILE_NAME, index=False, encoding='utf-8-sig')
    print(f"\n--- 5. 结果已保存至: {OUTPUT_FILE_NAME} ---")
    print("分析完成。")

# %%
# 执行主函数
if __name__ == '__main__':
    analyze_papers_bert(FILE_PATH)